<a href="https://colab.research.google.com/github/lalanikarim/notebooks/blob/main/Spoil_a_wish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spoil-A-Wish with LangChain, Cloudflare Workers AI, and Gradio

Spoil-A-Wish is a fun little game you play with friends where you make a wish and your friend grants it with a twist, essentially ruining it.

For example:

```
Wish: I wish for a million dollars.  
Response: Granted, but you get a million dollars in ZWL (Zimbabwe's defunct currency) which equals $15 USD. Enjoy your happy meal.
```

In this sample project, we will create a simple AI Agent that spoils your wishes.

We will build it with [LangChain](https://www.langchain.com) using [llama 3.1 70b instruct](https://developers.cloudflare.com/workers-ai/models/llama-3.1-70b-instruct/) model hosted on [Cloudflare Workers AI](https://developers.cloudflare.com/workers-ai/).  

`llama 3.1 70b instruct` model is available for free by Cloudflare while it is still in `beta` status. Other models are also available with [daily free token allocations](https://developers.cloudflare.com/workers-ai/platform/pricing/#free-allocation).

You will need your Cloudflare Account ID and API Token for this exercise.  
You can register for a free Cloudflare Account [https://www.cloudflare.com/](https://www.cloudflare.com/).

Account ID can be found at [https://dash.cloudflare.com](https://dash.cloudflare.com) under the `Overview` section towards the bottom. You may have to select a domain if you have multiple domains on your account.

API Token can be created [https://dash.cloudflare.com/profile/api-tokens](https://dash.cloudflare.com/profile/api-tokens). Make sure to give it permissions for `Workers AI`.

List of all availabe models supported by `Cloudflare Workers AI` can be found at [https://developers.cloudflare.com/workers-ai/models/](https://developers.cloudflare.com/workers-ai/models/).  
Select `Text Generation` under `Model Types`.

Detailed instructions for obtaining `Account Id`, `Account API Token`, and `Model Name` from Cloudflare can be found here: https://github.com/lalanikarim/spoil-a-wish/blob/main/Cloudflare.md

## 1. Install python dependencies

In [1]:
!pip install -q langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00


## 2. Import packages

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import BaseOutputParser
from langchain_community.chat_models.cloudflare_workersai import ChatCloudflareWorkersAI
import json
import ast

## 3. Extract Google Colab Secrets

In [3]:
import os
from google.colab import userdata

CLOUDFLARE_ACCOUNT_ID = userdata.get('CLOUDFLARE_ACCOUNT_ID')
CLOUDFLARE_API_TOKEN = userdata.get('CLOUDFLARE_API_TOKEN')

if not CLOUDFLARE_ACCOUNT_ID or not CLOUDFLARE_API_TOKEN:
    raise ValueError("CLOUDFLARE_ACCOUNT_ID and CLOUDFLARE_API_TOKEN must be set in environment variables or userdata.")

os.environ['CLOUDFLARE_ACCOUNT_ID'] = CLOUDFLARE_ACCOUNT_ID
os.environ['CLOUDFLARE_API_TOKEN'] = CLOUDFLARE_API_TOKEN

### Verify Cloudflare Account ID and API Key with Workers AI

In [4]:
!curl https://api.cloudflare.com/client/v4/accounts/$CLOUDFLARE_ACCOUNT_ID/ai/run/@cf/meta/llama-3.1-70b-instruct \
  -X POST \
  -H "Authorization: Bearer $CLOUDFLARE_API_TOKEN" \
  -d '{ "messages": [{ "role": "system", "content": "You are a friendly assistant" }, { "role": "user", "content": "Why is pizza so good" }]}'

{"result":{"response":"There are so many reasons why pizza is so good. Here are some possible explanations:\n\n1. **Combination of flavors**: Pizza combines a variety of flavors, from the savory sauce to the gooey cheese, and the various toppings you can add. The combination of these flavors creates a delicious and harmonious taste experience.\n2. **Crunchy crust**: A well-made pizza crust is crispy on the outside and fluffy on the inside. The crunchy texture provides a satisfying contrast to the soft toppings.\n3. **Melted cheese**: Let's face it, cheese is amazing, and melted cheese is even better! The melted cheese on pizza binds all the flavors together and adds a rich, creamy texture.\n4. **Variety of toppings**: Pizza allows you to customize with a wide range of toppings, from classic pepperoni and mushrooms to more adventurous options like pineapple and prosciutto. There's a topping combination out there for everyone!\n5. **Nostalgia**: For many people, pizza is a comfort food t

## 4. Prepare Prompt Template

In [5]:
# prompt: create a prompt template for a game of "Spoil a wish". In this game the user asks for a wish and the ai grants the wish but spoils it.

prompt_template = """
You are playing the game "Spoil a Wish".  A user will make a wish. Your job is to grant their wish, but then spoil it in a creative and humorous way.  Make sure to follow these rules:

1. First, acknowledge and grant the wish in a positive and enthusiastic manner.
2. Then, introduce a twist or complication that completely ruins or undermines the wish in a funny way. Be creative with the way the wish is ruined.
3. Your response should be concise and engaging.

Here is the wish:

{wish}
"""

prompt = PromptTemplate.from_template(prompt_template)

## 5. Initialize LLM and create chain

In [6]:
# prompt: Initialize an LLM from langchain_community.chat_models.cloudflare_workersai.ChatCloudflareWorkersAI. Use CLOUDFLARE_ACCOUNT_ID and CLOUDFLARE_API_TOKEN from google.colab.userdata

model = ChatCloudflareWorkersAI(account_id=os.getenv('CLOUDFLARE_ACCOUNT_ID'), api_token=os.getenv('CLOUDFLARE_API_TOKEN'),model='@cf/meta/llama-3.1-70b-instruct')

# Example usage (you can replace "I wish for a million dollars" with any wish)
wish = "I wish for a million dollars"

chat = prompt | model

### Test chain

In [7]:
response: AIMessage = chat.invoke({"wish": wish})
response.content

'{\'result\': {\'response\': "Your wish is GRANTED! You\'re now the proud owner of ONE MILLION DOLLARS!\\n\\nHowever, there\'s a tiny catch - it\'s all in pennies, and the pennies are stored in a giant vault that\'s located at the bottom of the world\'s largest ball pit, which is also infested with mischievous, penny-hoarding raccoons. Good luck getting to your newfound fortune without losing your mind or getting covered in raccoon prints!"}, \'success\': True, \'errors\': [], \'messages\': []}'

### Convert Cloudflare Workers AI response

In [8]:
ast.literal_eval(response.content)

{'result': {'response': "Your wish is GRANTED! You're now the proud owner of ONE MILLION DOLLARS!\n\nHowever, there's a tiny catch - it's all in pennies, and the pennies are stored in a giant vault that's located at the bottom of the world's largest ball pit, which is also infested with mischievous, penny-hoarding raccoons. Good luck getting to your newfound fortune without losing your mind or getting covered in raccoon prints!"},
 'success': True,
 'errors': [],
 'messages': []}

## 6. Create Custom Output Parser for Cloudflare Workers AI Response

In [9]:
from pydantic import BaseModel, Field
from typing import List

class CFWAIResult(BaseModel):
  response: str = Field(..., description="The response content.")

class CFWAIResponseMessage(BaseModel):
  success: bool = Field(description="Indicates if the operation was successful.")
  errors: List[str] = Field(default_factory=list, description="A list of errors encountered.")
  messages: List[str] = Field(default_factory=list, description="A list of informational messages.")
  result: CFWAIResult = Field(description="The result of the operation.")

class CFWAIResponseParser(BaseOutputParser):
    def parse(self, text: str) -> CFWAIResponseMessage:
        return CFWAIResponseMessage.parse_obj(ast.literal_eval(text))

### Create new chain with Custom Output Parser

In [10]:
chat = prompt | model | CFWAIResponseParser()

In [11]:
response: CFWAIResponseMessage = chat.invoke({"wish": wish})
print(response.result.response)

OH MY, A MILLION DOLLARS?! Your wish is GRANTED! You're now the proud owner of a million dollars, and you can buy all the fancy things your heart desires!

BUT, here's the catch: the million dollars is in the form of one million single dollar bills, and they're all in Zimbabwean dollars, which means they're equivalent to about 50 US cents in total! To make matters worse, the bills are all fluttering away in a giant blizzard, and you'll have to chase them down in the middle of a hilarious and chaotic money storm!


<ipython-input-9-3a73e571b6e9>:15: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return CFWAIResponseMessage.parse_obj(ast.literal_eval(text))


In [12]:
response.result.response

"OH MY, A MILLION DOLLARS?! Your wish is GRANTED! You're now the proud owner of a million dollars, and you can buy all the fancy things your heart desires!\n\nBUT, here's the catch: the million dollars is in the form of one million single dollar bills, and they're all in Zimbabwean dollars, which means they're equivalent to about 50 US cents in total! To make matters worse, the bills are all fluttering away in a giant blizzard, and you'll have to chase them down in the middle of a hilarious and chaotic money storm!"

## 7. Build a UI with Gradio

In [13]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 14.1 MB/s eta 0:00:00


In [14]:
import gradio as gr

def spoil_wish(wish):
    response: CFWAIResponseMessage = chat.invoke({"wish": wish})
    return response.result.response

with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as demo:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish")
            response = gr.TextArea(label="Response", visible=False)
            submit = gr.Button("Spoil Wish", elem_classes="btn success")
            reset = gr.Button("Try Again!", elem_classes="btn danger", visible=False)

    gr.Examples(
        examples=[
            ["I wish for a million dollars"],
            ["I wish for a new car"],
            ["I wish for a new house"],
            ["I wish for a new job"]
        ],
        inputs=[wish],
        label="Try these"

    )

    gr.on(
        triggers=submit.click,
        outputs=[wish, response, submit],
        fn=lambda: [gr.TextArea(interactive=False), gr.TextArea(visible=True), gr.Button(visible=False)],
        api_name=False
    ).then(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    ).then(
        outputs=reset,
        fn=lambda: gr.Button(visible=True),
        api_name=False
    )
    gr.on(
        triggers=reset.click,
        outputs=[response, reset, wish, submit],
        fn=lambda: [gr.TextArea(visible=False), gr.Button(visible=False), gr.TextArea(interactive=True, value=None), gr.Button(visible=True)],
        api_name=False
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://961115777a7fff7018.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
